In [ ]:
# Step 1: Install gTTS for voice
!pip install gTTS

# Step 2: Import libraries
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from gtts import gTTS
from IPython.display import Audio, display

# Step 3: Training text
text = "machine learning is powerful. deep learning is a branch of machine learning. artificial intelligence includes machine learning."

# Step 4: Tokenize words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
word_index = tokenizer.word_index
index_word = {v: k for k, v in word_index.items()}
vocab_size = len(word_index) + 1

# Step 5: Prepare sequences (2-word to 1-word)
words = text.lower().split()
sequences = []

for i in range(1, len(words)):
    seq = words[i-1:i+1]
    encoded = tokenizer.texts_to_sequences([' '.join(seq)])[0]
    if len(encoded) == 2:
        sequences.append(encoded)

sequences = np.array(sequences)
x, y = sequences[:, 0], sequences[:, 1]
x = x.reshape((x.shape[0], 1))
y = to_categorical(y, num_classes=vocab_size)

# Step 6: Build and train model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=1))
model.add(LSTM(64))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(x, y, epochs=300, verbose=0)

# Step 7: Input from user
input_word = input("Enter one word: ").lower()

if input_word not in word_index:
    print("❌ Word not found in training text!")
else:
    encoded = tokenizer.texts_to_sequences([input_word])[0]
    encoded = np.array(encoded).reshape(1, 1)

    pred = model.predict(encoded, verbose=0)
    next_index = np.argmax(pred)
    predicted_word = index_word.get(next_index, "<unk>")

    # Text output
    print("✅ Predicted next word:", predicted_word)

    # Voice output
    tts = gTTS(predicted_word)
    tts.save("predicted.mp3")
    display(Audio("predicted.mp3", autoplay=True))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Enter one word: machine


✅ Predicted next word: learning
